# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [7]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [8]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [9]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """

    #creo le copie su cui devo lavorare
    X_train = X_train.copy()
    X_val = X_val.copy()
    
    if hyperparams['use_pca']:
        if hyperparams['pca_standardize']:
            # standardizzo i dati
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)

        
        #Scrivo il codice per applicare la PCA
        pca=PCA(n_components=hyperparams['pca_components'])
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)    

    if hyperparams['data_standardize']:
        # ...
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Aggiunge il termine costante ai dati
    # aggiungo un vettore costante di uno
    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_val = np.c_[np.ones((X_val.shape[0], 1)), X_val]

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        # Calcolo della soluzione di regressione lineare con regolarizzazione L2
        lambda_ = hyperparams['reg_lambda']
        #Creo una matrice identità delle dimensioni pari al numero di colonne di X_train.
        I = np.eye(X_train.shape[1]) 

        #Si usa una matrice identità per regolarizzare i coefficienti della regressione, ma si mette a zero 
        #il primo elemento perché non si vuole regolarizzare l’intercetta.
        I[0,0] = 0

        X_trasposta = X_train.T  
        inversa = np.linalg.inv(X_trasposta @ X_train + lambda_ * I)  
        parametri = (inversa @ X_trasposta) @ y_train
  
    else:
        # ...
        X_trasposta = X_train.T  
        inversa = np.linalg.inv(X_trasposta @ X_train)  
        parametri = (inversa @ X_trasposta) @ y_train

    # Calcolo predizioni
    # ...
    y_train_pred = X_train @ parametri 
    y_val_pred = X_val @ parametri 

    # Calcola il MAE
    # ...
    mae_val=np.mean(np.abs(y_val-y_val_pred))
    mae_train=np.mean(np.abs(y_train-y_train_pred))

    return mae_val, mae_train, parametri

# Dividi il dataset in training e test set
# ...
train_porzione = 0.8 
test_porzione = 0.2

num_train = int(train_porzione * X.shape[0])
X_train = X[:num_train]
y_train = y[:num_train]

X_test = X[num_train:]
y_test = y[num_train:]

print("Train set: ", X_train.shape)
print("Test set: ", X_test.shape)

# Dividi il training set in training set effettivo e validation set
# ...

train_porzione_effettivo = 0.8 * train_porzione
val_porzione = 0.2 * train_porzione

num_train = int(train_porzione_effettivo * X.shape[0])
num_validation = int(val_porzione * X.shape[0])

X_train_ef = X[:num_train]
y_train_ef = y[:num_train]

X_validation = X[num_train:num_train+num_validation]
y_validation = y[num_train:num_train+num_validation]


print("Train set effettivo: ", X_train_ef.shape)
print("Validation set: ", X_validation.shape)


# Trova la configurazione di iperparametri migliore
# ...
migliore_config = None
migliore_val_mae = float('inf')
migliore_train_mae = None

for values in configs:
    MAE_val, MAE_train, _ = pipeline(X_train_ef, y_train_ef, X_validation, y_validation, values)
    if MAE_val < migliore_val_mae:
        migliore_val_mae = MAE_val
        migliore_train_mae = MAE_train
        migliore_config = values

print("Miglior configurazione di iperparametri trovata:")
print(migliore_config)
print("MAE validation:", migliore_val_mae)
print("MAE training:", migliore_train_mae)

# Riallena il modello sul training set completo
# ...
_, _, parametri_migliori = pipeline(X_train, y_train, X_test, y_test, migliore_config)

# Calcola il MAE sul test set
# ...
X_test_copy = X_test.copy()
if migliore_config['use_pca']:
    if migliore_config['pca_standardize']:
        X_train = scaler.pca.fit_transform(X_train)
        X_test_copy = scaler.pca.transform(X_test_copy)
    pca = PCA(n_components=migliore_config['pca_components'])
    X_train = pca.fit_transform(X_train)
    X_test_copy = pca.transform(X_test_copy)

if migliore_config['data_standardize']:
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test_copy = scaler.transform(X_test_copy)       
X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_copy = np.c_[np.ones((X_test_copy.shape[0], 1)), X_test_copy]


#predizioni sul test set
y_test_pred = X_test_copy @ parametri_migliori

mae_test = np.mean(np.abs(y_test - y_test_pred))

# Stampa  risultati
# ...
print(mae_test)

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Train set:  (3918, 11)
Test set:  (980, 11)
Train set effettivo:  (3134, 11)
Validation set:  (783, 11)
Miglior configurazione di iperparametri trovata:
{'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
MAE validation: 0.5735025699479851
MAE training: 0.6061022582434107
0.5481392209725904
